In [37]:
import numpy as np
import pickle
import os
from glob import glob
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [126]:
root = '/media/liamcli/fastfiles/results/icml2020'
plt.style.use('ggplot')
params = {'legend.fontsize': 14}
plt.rcParams.update(params)
plt.rcParams.update({'font.size': 12})

In [24]:
def get_arch_params(folder, softmax=False):
  files = glob(os.path.join(folder, 'one_shot_architecture*.obj'))
  ordered = []
  for f in files:
    start = f.find('architecture_') + len('architecture_')
    end = f.find('.obj') 
    epoch = int(f[start:end])
    ordered.append((epoch, f))
  ordered = sorted(ordered, key=lambda x:x[0])
  archs = []
  for e, f in ordered:
    with open(f, 'rb') as b:
      arch = pickle.load(b)
      if softmax:
        for i, p in enumerate(arch):
          normalized = np.exp(p)
          arch[i] = normalized / np.sum(normalized, axis=-1, keepdims=True)
          
      archs.append(arch)
  return archs

In [55]:
def entropy(p, axis):
  return -np.sum(p*np.log(p), axis=axis)

def get_entropy(history):
  result = {
    'operations': [],
    'edges': [],
  }
  for params in history:
    result['operations'].append(np.average(entropy(params[0],axis=-1)))
    result['edges'].append(np.average([entropy(p, axis=-1) for p in params[1:]]))
  return result
    
    
    

In [131]:
folders = {
  'pc_darts': [
    'ameet/search_space_1/search-pc_darts_space1_seed3500', 
    'ameet/search_space_2/search-pc_darts_space2_seed3501',
    'ameet/search_space_3/search-pc_darts_space3_seed3502'
  ],
  'epc_edarts':[
    'ameet/search_space_1/search-epc_edarts_space1_seed5100', 
    'ameet/search_space_2/search-epc_edarts_space2_seed5101',
    'ameet/search_space_3/search-epc_edarts_space3_seed5102'
  ]
}
styles = {
  'pc_darts': '-',
  'epc_edarts': '--'
}
colors = {
  'pc_darts': 'b',
  'epc_edarts': 'r'
}
labels = {
  'pc_darts': 'PC-DARTS',
  'epc_edarts': 'GAEA PC-DARTS',
  'darts': 'DARTS (first order)',
  'edarts': 'GAEA DARTS (bi-level)'
}
pdf = PdfPages('1shot1_entropy.pdf')
for s in [1, 2, 3]:
  for method in ['pc_darts', 'epc_edarts']:
    exp_dir = os.path.join(root, folders[method][s-1])
    data = get_arch_params(os.path.join(root, exp_dir+'*'), True if method=='pc_darts' else False)
    ent = get_entropy(data)
    labeled=False
    for f in ['operations']:
      if not labeled:
        plt.plot(ent[f], label=labels[method], linestyle=styles[method], color=colors[method])
        labeled=True
      else:
        plt.plot(ent[f], linestyle=styles[method], color=colors[method])
  plt.title('NAS-Bench-1Shot1: Space {} Entropy'.format(s))
  plt.xlim([0, 50])
  plt.xlabel('Epochs')
  plt.ylabel('Entropy')
  plt.legend()
  plt.tight_layout()
  pdf.savefig()
  plt.close()
pdf.close()
 

In [128]:
def get_history_entropy(folder, softmax=False):
  data = pickle.load(open(os.path.join(folder, 'history.pkl'), 'rb'))
  results = {
    'operations': [],
    'edges': []
  }
  for i, r in enumerate(data['alphas']):
    es = []
    for ct in r:
      prob = r[ct]
      if softmax:
        prob = np.exp(prob)
        prob = prob / np.sum(prob, axis=-1, keepdims=True)
      es.append(np.average(entropy(prob, axis=-1)))
    results['operations'].append(es)
  #if 'edges' in data:
  #  for i, r in enumerate(data['edges']):
  #    es = []
  #    for ct in r:
  #      offset = 0
  #      n_inputs = 2
  #      for n in range(4):
  #        es.extend(entropy(r[ct][offset:offset+n_inputs], axis=-1))
  #        offset += n_inputs
  #        n_inputs += 1
  #    results['edges'].append(np.average(es))
  return results
      
  
  

In [129]:
folders = {
  'epc_edarts': [
    'ameet/search-pcdarts-eedarts-cifar10-3103'
  ],
  'pc_darts':[
    'nina/search-pcdarts-darts-cifar10-3600'
  ]
}
styles = {
  'pc_darts': '-',
  'epc_edarts': '--'
}
colors = {
  'pc_darts': 'b',
  'epc_edarts': 'r'
}
pdf = PdfPages('darts_entropy.pdf')
for method in ['pc_darts', 'epc_edarts']:
  exp_dir = os.path.join(root, folders[method][0])
  #data = get_arch_params(os.path.join(root, exp_dir+'*'), True if method=='pc_darts' else False)
  ent = get_history_entropy(os.path.join(root, exp_dir), True if method=='pc_darts' else False)
  labeled=False
  for f in ent:
    if not labeled:
      plt.plot([t[0] for t in ent[f]], label=labels[method], linestyle=styles[method], color=colors[method])
      #plt.plot([t[1] for t in ent[f]], linestyle=styles[method], color=colors[method])
      labeled=True
    else:
      plt.plot(ent[f], linestyle=styles[method], color=colors[method])
plt.title('DARTS Search Space Entropy'.format(s))
plt.xlim([0, 50])
plt.xlabel('Epochs')
plt.ylabel('Entropy')
plt.legend()
plt.tight_layout()
pdf.savefig()
plt.close()
pdf.close()
 

In [130]:
root = '/media/liamcli/fastfiles/results/icml2020'
folders = {
  'darts': [
    'ameet/search-nas-bench-201-fdarts-cifar10-4400'
  ],
  'edarts':[
    'search-nas-bench-201-edarts-cifar10-6500'
  ]
}
styles = {
  'darts': '-',
  'edarts': '--'
}
colors = {
  'darts': 'b',
  'edarts': 'r'
}
pdf = PdfPages('nasbench201_entropy.pdf')
for method in ['darts', 'edarts']:
  exp_dir = os.path.join(root, folders[method][0])
  #data = get_arch_params(os.path.join(root, exp_dir+'*'), True if method=='pc_darts' else False)
  ent = get_history_entropy(os.path.join(root, exp_dir), True if method=='darts' else False)
  labeled=False
  for f in ent:
    if not labeled:
      plt.plot([t[0] for t in ent[f]], label=labels[method], linestyle=styles[method], color=colors[method])
      #plt.plot([t[1] for t in ent[f]], linestyle=styles[method], color=colors[method])
      labeled=True
    else:
      plt.plot(ent[f], linestyle=styles[method], color=colors[method])
plt.title('NAS-Bench-201:  CIFAR-10 Entropy'.format(s))
plt.xlim([0, 50])
plt.xlabel('Epochs')
plt.ylabel('Entropy')
plt.legend()
plt.tight_layout()
pdf.savefig()
plt.close()
pdf.close()
 